## Documentation

To read more about the search after parameter, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/8.15/paginate-search-results.html#search-after).

![search_after_docs](../images/search_after_docs.png)

## Connect to ElasticSearch


In [1]:
import { Client } from "npm:@elastic/elasticsearch";
import { load } from "https://deno.land/std/dotenv/mod.ts";

const env = await load({ envPath: "../.env" });

const client = new Client({
  node: env.ELASTICSEARCH_NODE,
  auth: {
    apiKey: env.ELASTICSEARCH_API_KEY,
  },
});

const info = await client.info();
console.log(info);


{
  name: "fead23d3120d",
  cluster_name: "docker-cluster",
  cluster_uuid: "v3fUyW9OReext6IjPiOCqg",
  version: {
    number: "8.17.4",
    build_flavor: "default",
    build_type: "docker",
    build_hash: "c63c7f5f8ce7d2e4805b7b3d842e7e792d84dda1",
    build_date: "2025-03-20T15:39:59.811110136Z",
    build_snapshot: false,
    lucene_version: "9.12.0",
    minimum_wire_compatibility_version: "7.17.0",
    minimum_index_compatibility_version: "7.0.0"
  },
  tagline: "You Know, for Search"
}


## Create index


In [2]:
await client.indices.delete({ index: "my_index", ignore_unavailable: true });
await client.indices.create({
  index: "my_index",
  mappings: {
    properties: {
      id: { type: "keyword" },
      value: { type: "float" },
      description: { type: "text" },
      category: { type: "keyword" },
      timestamp: { type: "date" },
    },
  },
});


{ acknowledged: true, shards_acknowledged: true, index: "my_index" }

Let's index the documents.


The documents will be duplicated to create a total of `100,000` documents. This is done to compare the `from/size` method with the `search_after` method.

In [3]:
import data from "../data/dummy_data_3.json" with { type: "json" };

type Document = {
  id?: string;
  timestamp?: string;
  value: number;
  category: string;
  description: string;
};

function generateBulkData(baseDocuments: Document[], targetSize = 100000): Document[] {
  const documents: Document[] = [];
  const baseCount = baseDocuments.length;
  const duplicationsNeeded = Math.floor(targetSize / baseCount);
  const baseTimestamp = new Date();

  for (let i = 0; i < duplicationsNeeded; i++) {
    for (const document of baseDocuments) {
      const newDoc: Document = { ...document };
      newDoc.id = `doc_${documents.length}`;
      newDoc.value = document.value + (Math.random() * 20 - 10); // random float between -10 and 10
      const timestamp = new Date(baseTimestamp.getTime() - documents.length * 60 * 1000);
      newDoc.timestamp = timestamp.toISOString();
      documents.push(newDoc);
    }
  }

  return documents;
}

const documents = generateBulkData(data, 100000);

console.log(documents.length);


100000


Since, we have duplicated the dummy data so much. Let's use the `bulk API` since we learned it before to index all those documents rapidly.


In [4]:
const operations = [];
for (const document of documents) {
  operations.push({
    index: {
      _index: "my_index",
    },
  });
  operations.push(document);
}

await client.bulk({
  operations,
});


{
  errors: false,
  took: 1817,
  items: [
    {
      index: {
        _index: "my_index",
        _id: "voccGpYBkrY7cs0F4otB",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 0,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "v4ccGpYBkrY7cs0F4otB",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 1,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "wIccGpYBkrY7cs0F4otB",
        _version: 1,
        result: "created",
        _shards: { total: 2, successful: 1, failed: 0 },
        _seq_no: 2,
        _primary_term: 1,
        status: 201
      }
    },
    {
      index: {
        _index: "my_index",
        _id: "wYccGpYBkrY7cs0F4otB",
        _version: 1,
        result: "created",
      

## From / Size method

To use the `from/size` method, include two parameters in your query: `from`, which specifies the number of documents to skip, and `size`, which tells Elasticsearch how many documents to return.

In [15]:
const response = await client.search({
  index: "my_index",
  from: 0,
  size: 10,
  sort: {
    timestamp: "desc",
    id: "desc",
  },
});

const hits = response.hits.hits;
for (const hit of hits) {
  console.log(hit._source.id);
}


doc_0
doc_1
doc_2
doc_3
doc_4
doc_5
doc_6
doc_7
doc_8
doc_9


To retrieve the next batch of documents, adjust the `from` parameter from 0 to 10.

In [16]:
const response = await client.search({
  index: "my_index",
  from: 10, // Adjust here
  size: 10,
  sort: {
    timestamp: "desc",
    id: "desc",
  },
});

const hits = response.hits.hits;
for (const hit of hits) {
  console.log(hit._source.id);
}


doc_10
doc_11
doc_12
doc_13
doc_14
doc_15
doc_16
doc_17
doc_18
doc_19


## Search after method

To use the `search_after` method, include the following parameters in your query:

1. **size**: Specifies the number of documents to retrieve in each batch, similar to the `size` parameter in `from/size`.

2. **sort**: The `search_after` method requires specifying one or more fields to sort the results, such as `timestamp` or `id`. Sorting ensures a consistent order for navigating through result pages.

In [17]:
const response = await client.search({
  index: "my_index",
  from: 0,
  size: 10,
  sort: {
    timestamp: "desc",
    id: "desc",
  },
});

const hits = response.hits.hits;
for (const hit of hits) {
  console.log("Document ID:", hit._source.id);
  console.log("Value:", hit._source.value);
}


Document ID: doc_0
Value: 19.928982280872738
Document ID: doc_1
Value: 108.96828524704071
Document ID: doc_2
Value: 8.22266896005628
Document ID: doc_3
Value: 155.40614116869534
Document ID: doc_4
Value: 31.109293462253152
Document ID: doc_5
Value: 7.925037262386985
Document ID: doc_6
Value: 50.848438746922334
Document ID: doc_7
Value: 307.0562204352602
Document ID: doc_8
Value: 10.419580785486556
Document ID: doc_9
Value: 73.20593164852329


To retrieve the next batch of documents using `search_after`, you’ll pass the `sort` values from the last document of the previous batch to the `search_after` parameter in the subsequent query.

In [18]:
const lastSortValue = hits[hits.length - 1].sort;

const response = await client.search({
  index: "my_index",
  size: 10,
  sort: {
    timestamp: "desc",
    id: "desc",
  },
  search_after: lastSortValue,
});

const hits = response.hits.hits;
for (const hit of hits) {
  console.log("Document ID:", hit._source.id);
  console.log("Value:", hit._source.value);
}


Document ID: doc_10
Value: 12.231060483336362
Document ID: doc_11
Value: 107.6631114805679
Document ID: doc_12
Value: 5.641338290160385
Document ID: doc_13
Value: 155.62636621694477
Document ID: doc_14
Value: 21.497208772289675
Document ID: doc_15
Value: -3.1885614726155262
Document ID: doc_16
Value: 58.331152202624835
Document ID: doc_17
Value: 302.6084141960815
Document ID: doc_18
Value: 19.07907910498851
Document ID: doc_19
Value: 62.73603487529942


## Conclusion

For larger indexes, it's recommended to use the `search_after` method. For smaller indexes, both methods work well.